In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../")
sys.path.append("../..")

import json
import MSOutlierDetection
import MissingPatternDetection
import Imputation
import matplotlib.pyplot as plt 


class partialDataProcessing():
    def __init__(self):
        self.columnNaNCount={}
        self.columnNaNRatio={}
        
    def setData(self, data):
        self.data = data
        self.totalLength = len(data)
        self.columns = data.columns
    
    # 옆에서 가져왔음, 수정해야함 
    def get_preprocessed_data(self, data_raw, clean_param):
        if clean_param['flag'] ==True:
            from KETIPrePartialDataPreprocessing.PartialDataCleansing.definite_error_detection import min_max_limit_value
            self.limit_min_max = min_max_limit_value.MinMaxLimitValueSet().get_data_min_max_limitSet(clean_param['data_type'])
            preprocessed_data = MSOutlierDetection.CertainOutlierDetection().get_valid_data(data_raw, self.limit_min_max)
            #preprocessed_data = MSOutlierDetection.uncertain_outlier_detection(preprocessed_data, column) 
            # Column Error
        else:
            preprocessed_data = data_raw.copy()
        return preprocessed_data
    
    def dataProcessingByColumn(self, imputation_parameter):
        #Outlier2NaN = OutlierDetection.CertainOutlierDetection()
        clean_param={'flag':True, 'data_type':'air'}
        self.dataWithMoreNaN = self.get_preprocessed_data(self.data, clean_param)
        # 1. Measurement Outlier Detection Module`
        
        for column in self.columns:
            column_data = self.dataWithMoreNaN[[column]]
           
            # 2. Missing Pattern Detection Module
            NaNPatternCheck = MissingPatternDetection.MissingPatternDetection()
            column_data = NaNPatternCheck.get_missing_pattern(column_data, column)
            
            # 3. Missing Data Imputation
            imputation_method = imputation_parameter['imputation_method']
            totalNanLimit = imputation_parameter['totalNanLimit']
        
            self.columnNaNCount[column]=self.data[column].isna().sum()
            self.columnNaNRatio[column]=round(float(self.columnNaNCount[column]*100/self.totalLength), 2)
            if (self.columnNaNRatio[column] < totalNanLimit):
            # if total column NaN number is less tan limit, Impute it according to the parameter    
                for method_set in imputation_method:
                    # 3. Missing Data Imputation
                    column_data = self.imputeColumnData(method_set, column_data)
    
    #def outlierToNaN(self, data)
    def imputeColumnData(self, method_set, dataset):
        min_limit = method_set['min']
        max_limit = method_set['max']
        method = method_set['method']
        Imputer = Imputation.imputation_methods()
        if method == 'mean':
            dataset = Imputer.mean_interpolate(dataset, min_limit, max_limit)

        elif method == 'median':
            dataset = Imputer.median_interpolate(dataset, min_limit, max_limit)

        elif method == 'bfill':
            dataset = Imputer.bfill(dataset, min_limit, max_limit)

        elif method == 'ffill':    
            dataset = Imputer.ffill(dataset,  min_limit, max_limit)

        if method == 'linear':
            dataset = Imputer.linear_interpolate(dataset,  min_limit, max_limit)
        
        elif method == 'time':
            dataset = Imputer.time_interpolation(dataset,  min_limit, max_limit)

        elif method == 'nearest':
            dataset = Imputer.nearest_interpolate(dataset, min_limit, max_limit)

        elif method == 'zero':
            dataset = Imputer.zero_interpolate(dataset, min_limit, max_limit)

        elif method == 'slinear':
            dataset = Imputer.slinear_interpolate(dataset, min_limit, max_limit)

        elif method == 'quadratic':
            dataset = Imputer.quadratic_interpolate(dataset, min_limit, max_limit)

        elif method == 'cubic':
            dataset = Imputer.cubic_interpolate(dataset, min_limi, max_limit)
    
        elif method == 'spline':
            dataset = Imputer.spline_interpolate(dataset,  min_limi, max_limit)

        elif method == 'barycentric':
            dataset = Imputer.barycentric_interpolate(dataset, min_limi, max_limit)

        elif method == 'polynomial':
            dataset = Imputer.polynomial_interpolate(dataset, min_limi, max_limit)
        return dataset
            
        
        
class getData():
    def getInfluxDB(self):
        pass
    
    def getFileInput(self, file_name, time_index="timedate"):
        dataset = pd.read_csv(file_name, parse_dates=True, index_col=[time_index])
        return dataset
    


In [2]:
input_file = './data_miss_original.csv'
imputation_parameter ={
   "imputation_method":[
      {
         "min":0,
         "max":1,
         "method":"mean"
      },
      {
         "min":2,
         "max":4,
         "method":"linear"
      },
      {
         "min":5,
         "max":10,
         "method":"brits"
      }
   ],
   "totalNanLimit":0.3
}

In [3]:
dataset = getData().getFileInput(input_file, 'timedate')
MDP = partialDataProcessing()
MDP.setData(dataset[:10000])
MDP.dataProcessingByColumn(imputation_parameter)





In [4]:
dataset.columns

Index(['temp', 'co2', 'pm10'], dtype='object')

In [5]:
for column in dataset.columns:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-21d3b0a1aecc>, line 2)